<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/models/studio_0/DDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna

In [ ]:
import pickle
import pandas as pd
import optuna
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
path="/content/drive/Shareddrives/Deep Learning/datasets/DDI/"

In [ ]:
dev = pd.read_csv(path+'ddi2013-type/dev.tsv', sep='\t')
test = pd.read_csv(path+'ddi2013-type/test.tsv', sep='\t')
train = pd.read_csv(path+'ddi2013-type/train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"ddi2013-type/DDI_sinonimi.csv")
data_embedding = pd.read_csv(path+"ddi2013-type/DDI_embedding.csv")

In [ ]:
print(train.shape)
print(data_sinonimi.shape)
print(data_embedding.shape)

In [ ]:
print(train.label.value_counts())
print(data_sinonimi.label.value_counts())
print(data_embedding.label.value_counts())

In [ ]:
with open(path+"word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Set up per allenamento Modello

In [ ]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]

my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

TRAINSET

In [ ]:
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_train = [my_text_to_word_sequence(sentence) for sentence in train['sentence']]

DEVSET

In [ ]:
five_hot_dev = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  dev.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_dev = [my_text_to_word_sequence(sentence) for sentence in dev['sentence']]

SINONIMI

In [ ]:
five_hot_sin = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_sinonimi.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

EMBEDDING

In [ ]:
five_hot_emb = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_embedding.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [ ]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_dev):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
  
print(f'Il massimo è {max}')

Il massimo è 92


Crao i vari embedding per tutti i dataset, quest'operazione e pesante 

In [ ]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
embedded_devset = np.zeros(shape=(len(sentences_dev), max, 300))
for i, sentence in enumerate(sentences_dev):
  for j, word in enumerate(sentence):
    try:
      embedded_devset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
  for j, word in enumerate(sentence):
    try:
      embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
  for j, word in enumerate(sentence):
    try:
      embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

Carica optuna results e inizializza il modello, oppure salva il modello oppure carica solo i pesi del modello 

In [ ]:
best_params = optuna.load_study(study_name="DDI", storage="sqlite:///"+path+"ddi2013-type/optuna_ddi_studio_0.db").best_params

In [ ]:
print(f'{best_params}')

{'batch_size': 89, 'dropout': 0.63, 'units': 81}


In [ ]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params['units'],
                                                             recurrent_dropout=best_params['dropout'],
                                                             activation='tanh')))

model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
#train the model
result = model.fit(embedded_trainset,
                   five_hot_train,
                   epochs=100,
                   batch_size=best_params['batch_size'],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

In [ ]:
#save the model
model.save_weights(path+'ddi2013-type/DDI_w_studio_0.h5')

In [ ]:
#load only the w of the model, the model must be already executed
model.load_weights(path+'ddi2013-type/DDI_w_studio_0.h5')

EVALUATE ALL DATASET

In [ ]:
result_base=model.evaluate(embedded_trainset,five_hot_train,batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

211/211 [==============================] - 8s 27ms/step - loss: 0.0788 - accuracy: 0.9739 </br>
DATASET ORIGINARIO[0.0771881714463234, 0.975557804107666]

In [ ]:
result_base=model.evaluate(embedded_devset,five_hot_dev,batch_size=best_params['batch_size'],)
print(f'DATASET DEV{result_base}')

82/82 [==============================] - 3s 26ms/step - loss: 0.5059 - accuracy: 0.8493 </br>
DATASET DEV[0.6515974402427673, 0.749309778213501]

In [ ]:
result_base=model.evaluate(embedded_sin,five_hot_sin,batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

211/211 [==============================] - 8s 27ms/step - loss: 0.4900 - accuracy: 0.8688 </br>
DATASET SINONIMI[0.464800626039505, 0.8818361163139343]

In [ ]:
result_base=model.evaluate(embedded_emb,five_hot_emb,batch_size=best_params['batch_size'],)
print(f'DATASET EMBEDDING{result_base}')

211/211 [==============================] - 6s 27ms/step - loss: 0.3109 - accuracy: 0.9032 </br>
DATASET EMBEDDING[0.29793423414230347, 0.9097928404808044]